# Apply spectral counting-NSAF feature on the detected peptides


In [ ]:
# import libraries
import numpy as np
import pandas as pd
import re
import csv

# set display options
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import datasets
fasta_peptides = pd.read_table('/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/fasta_peptides.tsv')
detected_peptides = pd.read_table('/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/detected_peptides.tsv')

In [ ]:
fasta_peptides.shape

(209525, 6)

In [ ]:
detected_peptides.shape

(11015, 3)

In [ ]:
# sort detected_peptides by PEP
detected_peptides = detected_peptides.sort_values(by=['PEP']).reset_index(drop = True)

In [ ]:
# remove any peptides in detected_peptides that map to more than one different protein
detected_peptides = detected_peptides.groupby('Peptide').filter(lambda x: x['Protein'].nunique() == 1)
detected_peptides.shape
# it seems no peptides are shared by more than one different protein, as expected (since we removed ambiguous protein groups)

(11015, 3)

In [ ]:
detected_peptides.describe(include="all")

,Protein,Peptide,PEP
count,11015,11015,11015.000000
unique,1104,1917,NaN
top,Q09666,DKFPGEFMK,NaN
freq,269,44,NaN
mean,NaN,NaN,0.235684
std,NaN,NaN,0.370207
min,NaN,NaN,0.004787
25%,NaN,NaN,0.004803
50%,NaN,NaN,0.006038
75%,NaN,NaN,0.368180


In [ ]:
fasta_peptides.describe(include="all")

,Protein,Peptide,Cleavage,Prediction,Probability,Length
count,209525,209525,209525.000000,209525.0,209525.000000,209525.000000
unique,4877,32597,NaN,NaN,NaN,NaN
top,P35579,K,NaN,NaN,NaN,NaN
freq,957,15701,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.129221,1.0,0.735361,830.453569
std,NaN,NaN,0.335445,0.0,0.118938,748.536186
min,NaN,NaN,0.000000,1.0,0.500000,0.000000
25%,NaN,NaN,0.000000,1.0,0.636000,389.000000
50%,NaN,NaN,0.000000,1.0,0.779000,621.000000
75%,NaN,NaN,0.000000,1.0,0.816000,1031.000000


## Perform spectral counting on detected_peptides

In [ ]:
fasta_protein_len_dict = dict(zip(fasta_peptides.Protein, fasta_peptides.Length))

In [ ]:
# map protein sequence lengths to each protein in detected_peptides
detected_peptides['Protein_length'] = detected_peptides['Protein'].map(fasta_protein_len_dict)
detected_peptides

,Protein,Peptide,PEP,Protein_length
0,P98179,YYDSRPGGYGYGYGR,0.004787,157.0
1,P30084,NNTVGLIQLNRPK,0.004787,290.0
2,P62851,DKLNNLVLFDK,0.004787,NaN
3,P62851,DKLNNLVLFDK,0.004787,NaN
4,P30084,NNTVGLIQLNRPK,0.004787,290.0
...,...,...,...,...
11010,Q02539,GGVSLAALKK,1.000000,NaN
11011,Q9P219,EKLHDVDFYK,1.000000,NaN
11012,Q9P219,EKLHDVDFYK,1.000000,NaN
11013,Q8IWA0,EKPDIFQLVSVK,1.000000,830.0


In [ ]:
# check for any NaN values in "Length" column
detected_peptides.isnull().sum()

Protein              0
Peptide              0
PEP                  0
Protein_length    3055
dtype: int64

In [ ]:
# assign a count value of '1' for each PSM (peptide spectrum match)
detected_peptides["PSM"] = 1
detected_peptides

,Protein,Peptide,PEP,Protein_length,PSM
0,P98179,YYDSRPGGYGYGYGR,0.004787,157.0,1
1,P30084,NNTVGLIQLNRPK,0.004787,290.0,1
2,P62851,DKLNNLVLFDK,0.004787,NaN,1
3,P62851,DKLNNLVLFDK,0.004787,NaN,1
4,P30084,NNTVGLIQLNRPK,0.004787,290.0,1
...,...,...,...,...,...
11010,Q02539,GGVSLAALKK,1.000000,NaN,1
11011,Q9P219,EKLHDVDFYK,1.000000,NaN,1
11012,Q9P219,EKLHDVDFYK,1.000000,NaN,1
11013,Q8IWA0,EKPDIFQLVSVK,1.000000,830.0,1


In [ ]:
# count number of times each PSM per protein appears
detected_peptides_abundance = detected_peptides.groupby("Protein")["PSM"].sum()
detected_peptides_abundance

Protein
A0A075B6V9     1
A0A087WTZ8     5
A0A0A0MR85     4
A0A0A0MRV7    16
A0A0A6YYC0     3
              ..
Q9Y6U3-2       5
Q9Y6Y8         5
R4GMQ2         7
S4R3H4         1
X6RCZ8        12
Name: PSM, Length: 1104, dtype: int64

In [ ]:
# map protein counts to detected_peptides
detected_peptides_abundance = pd.merge(detected_peptides, detected_peptides_abundance, on='Protein', how='left',
                                 suffixes=(None, '_per_protein'), indicator=True) \
                          .query("_merge == 'both'") \
                          .drop('_merge', 1) \
                          .drop('PSM', 1)
detected_peptides_abundance

<ipython-input-16-560c6768c5d9>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .drop('_merge', 1) \
<ipython-input-16-560c6768c5d9>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .drop('PSM', 1)


,Protein,Peptide,PEP,Protein_length,PSM_per_protein
0,P98179,YYDSRPGGYGYGYGR,0.004787,157.0,7
1,P30084,NNTVGLIQLNRPK,0.004787,290.0,29
2,P62851,DKLNNLVLFDK,0.004787,NaN,10
3,P62851,DKLNNLVLFDK,0.004787,NaN,10
4,P30084,NNTVGLIQLNRPK,0.004787,290.0,29
...,...,...,...,...,...
11010,Q02539,GGVSLAALKK,1.000000,NaN,8
11011,Q9P219,EKLHDVDFYK,1.000000,NaN,7
11012,Q9P219,EKLHDVDFYK,1.000000,NaN,7
11013,Q8IWA0,EKPDIFQLVSVK,1.000000,830.0,13


In [ ]:
# calculate spectral count (using NSAF approach) for detected_peptides
detected_peptides_abundance["Quantification"] = detected_peptides_abundance["PSM_per_protein"] / detected_peptides_abundance["Protein_length"]
detected_peptides_abundance

,Protein,Peptide,PEP,Protein_length,PSM_per_protein,Quantification
0,P98179,YYDSRPGGYGYGYGR,0.004787,157.0,7,0.044586
1,P30084,NNTVGLIQLNRPK,0.004787,290.0,29,0.100000
2,P62851,DKLNNLVLFDK,0.004787,NaN,10,NaN
3,P62851,DKLNNLVLFDK,0.004787,NaN,10,NaN
4,P30084,NNTVGLIQLNRPK,0.004787,290.0,29,0.100000
...,...,...,...,...,...,...
11010,Q02539,GGVSLAALKK,1.000000,NaN,8,NaN
11011,Q9P219,EKLHDVDFYK,1.000000,NaN,7,NaN
11012,Q9P219,EKLHDVDFYK,1.000000,NaN,7,NaN
11013,Q8IWA0,EKPDIFQLVSVK,1.000000,830.0,13,0.015663


In [ ]:
# check if any detected peptides are not in fasta_peptides (should be 0)
len(detected_peptides_abundance[~detected_peptides_abundance["Peptide"].isin(fasta_peptides["Peptide"])])

10355

In [ ]:
detected_peptides_abundance[~detected_peptides_abundance["Peptide"].isin(fasta_peptides["Peptide"])]
# after a random spot check these peptides seem to be present in fasta peptides, but a Methionine amino acid comes before it

,Protein,Peptide,PEP,Protein_length,PSM_per_protein,Quantification
0,P98179,YYDSRPGGYGYGYGR,0.004787,157.0,7,0.044586
2,P62851,DKLNNLVLFDK,0.004787,NaN,10,NaN
3,P62851,DKLNNLVLFDK,0.004787,NaN,10,NaN
6,P82673,NMEINKEELLGTK,0.004787,323.0,7,0.021672
7,Q14566,NLYHNLCTSLFPTIHGNDEVKR,0.004787,821.0,42,0.051157
...,...,...,...,...,...,...
11009,Q9P219,EKLHDVDFYK,1.000000,NaN,7,NaN
11010,Q02539,GGVSLAALKK,1.000000,NaN,8,NaN
11011,Q9P219,EKLHDVDFYK,1.000000,NaN,7,NaN
11012,Q9P219,EKLHDVDFYK,1.000000,NaN,7,NaN


In [ ]:
detected_peptides_abundance.describe(include="all")

,Protein,Peptide,PEP,Protein_length,PSM_per_protein,Quantification
count,11015,11015,11015.000000,7960.000000,11015.000000,7960.000000
unique,1104,1917,NaN,NaN,NaN,NaN
top,Q09666,DKFPGEFMK,NaN,NaN,NaN,NaN
freq,269,44,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.235684,867.241709,31.923831,0.069197
std,NaN,NaN,0.370207,1119.478199,48.205871,0.115614
min,NaN,NaN,0.004787,80.000000,1.000000,0.000393
25%,NaN,NaN,0.004803,284.000000,8.000000,0.019608
50%,NaN,NaN,0.006038,521.000000,15.000000,0.042802
75%,NaN,NaN,0.368180,911.000000,35.000000,0.073379


In [ ]:
# drop any duplicate peptides (i.e. several instances of same peptides that map to same protein) but keep first occurrence
detected_peptides_abundance = detected_peptides_abundance.drop_duplicates(subset = ['Peptide', 'Protein'],
                                                                          keep = "first").reset_index(drop = True)
detected_peptides_abundance.describe(include="all")

,Protein,Peptide,PEP,Protein_length,PSM_per_protein,Quantification
count,1917,1917,1917.000000,1344.000000,1917.000000,1344.000000
unique,1104,1917,NaN,NaN,NaN,NaN
top,Q09666,YYDSRPGGYGYGYGR,NaN,NaN,NaN,NaN
freq,40,1,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.011226,877.968006,25.293166,0.047458
std,NaN,NaN,0.025251,1078.848877,44.137130,0.076706
min,NaN,NaN,0.004787,80.000000,1.000000,0.000393
25%,NaN,NaN,0.004788,314.000000,5.000000,0.012469
50%,NaN,NaN,0.004826,546.500000,11.000000,0.029112
75%,NaN,NaN,0.005565,940.750000,25.000000,0.057656


In [ ]:
detected_peptides_abundance.shape

(1917, 6)

In [ ]:
# export dataset
detected_peptides_abundance.to_csv("/content/drive/MyDrive/Colab Notebooks/peptide ml detection/data/detected_peptides_NSAF.tsv", sep='\t', index=False)